## MongoDB連線 & 匯入jieba套件

In [3]:
import re
from pymongo import MongoClient 

#預設就是自己
client = MongoClient('127.0.0.1', 27017)
database = client['test']
collection =database['news']
client.close()

import jieba
jieba.set_dictionary('D:/TextMining/dict.txt.big.txt')  #切換至中文繁體字庫
jieba.load_userdict("D:/TextMining/dict_keyw.txt")       #加入自建詞庫
jieba.load_userdict("D:/TextMining/ptt.txt")       #加入PTT詞庫

Building prefix dict from D:\TextMining\dict.txt.big.txt ...
Loading model from cache c:\users\ytchen\appdata\local\temp\jieba.ufb55ea6623e696143f43e248198acd74.cache
Loading model cost 1.379 seconds.
Prefix dict has been built succesfully.


## 設定查詢條件

In [5]:
#給關鍵字----------------------------------------
tag1 = "林全"
# tag2 = "台灣"
#-----------------------------------------------

date=[]
title=[]
content =[]
all_article = []
#把資料庫東西抓出來
for post in collection.find(
    {"$and":[                   
            {"content":{"$regex":tag1}},
#             {"content":{"$regex":tag2}},
            #{"date":{"$regex":"2016"}},
            ]},{"_id":0}): 
    summary = post['content']
    all_article.append(summary)
    content.append(' '.join(jieba.cut(summary)))
    title.append(post['title'])
    date.append(post['date'])
    
client.close()

print title[0]

#文章數
newsNumber = len(title)
print "文章數:" + str(newsNumber)

名單公布-查黨產委員-被酸「綠油精」藍營人士不願來「政黨平衡有困難」
文章數:576


In [ ]:
def computingTFIDF(tfidf_content):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(content)  # titile 放文本
    weight = X.toarray() #裝關鍵字的權重值
    features = vectorizer.get_feature_names()     # 拿到所有的關鍵詞  
    return features,weight

## TF-IDF權重計算

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(content)  # titile 放文本
weight = X.toarray() #裝關鍵字的權重值

#特徵值總共有多少個
features = vectorizer.get_feature_names()     # 拿到所有的關鍵詞  
print "特徵值數量:",len(features)

特徵值數量: 2656


In [ ]:
def computingTFIDF(tfidf_content):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(content)  # titile 放文本
    weight = X.toarray() #裝關鍵字的權重值
    features = vectorizer.get_feature_names()     # 拿到所有的關鍵詞  
    return features,weight

def getTopTFIDF(features,weight):
    top_features = []
    final_top_features = []
    for n in range(0,newsNumber):  #迴圈參考上面的總文章數
        #裝每天文章的feature & score
        map_top_features = []

        #對特徵詞的index進行排序
        indices = np.argsort(weight[n])[::-1] #weight[n] 是第幾篇新聞的意思

        #拿到所有的特徵詞
        features = vectorizer.get_feature_names()

        #截取TOP多少的詞
        top_n = 40

        ###################################把每篇的TOP TF-IDF 與其分數存起來###############################################
        #將每天文章的TOP-TFIDF存起來
        top_features.append([features[i] for i in indices[:top_n]])

        a=0
        for i in top_features[n]:
    #         print i,weight[n][indices[a]]
            data = {
                'feature':i,
                'score':weight[n][indices[a]]
            }
            #print data['score'],data['feature']
            a=a+1

            map_top_features.append(data)
        final_top_features.append(map_top_features)
    return final_top_features
    

## 文章摘要-資料格式

In [2]:
#裝一篇文章的 TOP50 特徵詞 & 特徵詞的分數
map_top_features = [
    {
        'feature':i,
        'score':weight[n][indices[a]]
    }
] 

#裝計算後的結果 → 用於文章摘要計算
top_features = [
    map_top_features,
    map_top_features,
    map_top_features,
]

#裝句子與句子的分數
sen_score_map = {
    'sentence' : sen,
    'score' : score,
}

SyntaxError: invalid syntax (<ipython-input-2-1197b8aa4d1c>, line 1)

## 拿到每篇文章的TOP TF-IDF

In [25]:
import time
tic = time.clock()

top_features = []
final_top_features = []
for n in range(0,newsNumber):  #迴圈參考上面的總文章數
    #裝每天文章的feature & score
    map_top_features = []
    
    #對特徵詞的index進行排序
    indices = np.argsort(weight[n])[::-1] #weight[n] 是第幾篇新聞的意思
    
    #拿到所有的特徵詞
    features = vectorizer.get_feature_names()
    
    #截取TOP多少的詞
    top_n = 40
    
    ###################################把每篇的TOP TF-IDF 與其分數存起來###############################################
    #將每天文章的TOP-TFIDF存起來
    top_features.append([features[i] for i in indices[:top_n]])
    
    a=0
    for i in top_features[n]:
#         print i,weight[n][indices[a]]
        data = {
            'feature':i,
            'score':weight[n][indices[a]]
        }
        #print data['score'],data['feature']
        a=a+1
        
        map_top_features.append(data)
    final_top_features.append(map_top_features)
# print final_top_features[0]
    #############################################################################################
    
toc = time.clock()
print "執行時間:",(toc - tic),"秒"

執行時間: 0.183006105489 秒


In [ ]:
def getTopTFIDF(features,weight):
    top_features = []
    final_top_features = []
    for n in range(0,newsNumber):  #迴圈參考上面的總文章數
        #裝每天文章的feature & score
        map_top_features = []

        #對特徵詞的index進行排序
        indices = np.argsort(weight[n])[::-1] #weight[n] 是第幾篇新聞的意思

        #拿到所有的特徵詞
        features = vectorizer.get_feature_names()

        #截取TOP多少的詞
        top_n = 40

        ###################################把每篇的TOP TF-IDF 與其分數存起來###############################################
        #將每天文章的TOP-TFIDF存起來
        top_features.append([features[i] for i in indices[:top_n]])

        a=0
        for i in top_features[n]:
    #         print i,weight[n][indices[a]]
            data = {
                'feature':i,
                'score':weight[n][indices[a]]
            }
            #print data['score'],data['feature']
            a=a+1

            map_top_features.append(data)
        final_top_features.append(map_top_features)
    return final_top_features

## 文章摘要運算

In [26]:
#對所有的新聞文章進行 "句子切詞 "
import re
for article_index in range(0,newsNumber):
    article = all_article[article_index] #一篇文章
    setenceList = re.split("，|。|",article.encode('utf-8')) #根據"逗號" & "句號" 進行句子斷開
    
    #裝一篇文章的句子 & 句子的分數
    sen_score_map = []
    article_score = 0 #文章的總分數
    avg_score = 0 #總文章的平均分數
    
    #將每篇文章的句子逐一取出進行分數計算
    for sen in setenceList:
        #初始化句子分數
        score = 0 
        
        #逐一抓出各文章的 map_top_features 特徵詞與分數，與句子進行比對，若該特徵詞存在於句子中，則分數加上去
        for i in final_top_features[article_index]:
            feature = i['feature'].encode('utf-8')
            if feature in sen:
                score += i['score']
        
        #存取句子與分數
        data = {
            'sentence' : sen,
            'score' : score,
        }
        sen_score_map.append(data)
        
        #文章總分數
        article_score += score
        
    #文章資訊
    article_sen_size = len(setenceList) #文章句子總數
    avg_score = article_score / article_sen_size #平均句子分數
    
    #文章摘要
    articleSummarize = ''
    skip = 0
    for i in sen_score_map:
        #第一句一定要有，加完就continue換下一個迴圈，不用跑下面的判斷
        if skip == 0:
            skip+=1
            articleSummarize += '，'+i['sentence']
            continue
        #進行判斷(句子分數 > 平均分數才加入摘要)
        if i['score'] > avg_score*1.5:
            articleSummarize += '，'+i['sentence']
    #修整文章摘要-最後面的逗號改成句號
    articleSummarize = articleSummarize.replace('，','',1)+"。" #把最前面的逗號替換掉
            
    print '平均分數=' + str(avg_score)
    print '---------------------------------------------'
    print '新聞標題：'+title[article_index].encode('utf-8')
    print '------------'
    print '文章摘要：'
    print articleSummarize
    print '------------'
    print '文章原文：'
    print article
    print '---------------------------------------------'    
    

平均分數=0.25906203997
---------------------------------------------
新聞標題：不想上學？國一新生跳樓亡
------------
文章摘要：
12歲鍾姓少年疑因擔心上國中後課業壓力變大、不想上學，鐘父抱著兒子遺體痛哭失聲，即將進入國中校園的鍾姓少年，央請妹妹和他一起到頂樓，鍾姓少年隨即爬上女兒牆，妹妹見狀大吃一驚趕緊問他：「你想幹嘛？」少年卻回答：「我想自殺」，不要跳下去！」少年回答：「妳說的是原因之一」，對於兒子選擇以輕生來拒絕開學，兒子回家後還主動幫忙掃地、拖地，兒子雖曾在學校被同學取笑是低收入戶，他安慰兒子好好念書，此後兒子都沒有出異狀，外傳鍾父一早與兒子爭執進而引發輕生，兒子國小畢業後，也還有數個國小的要好同學與他同校；前晚就寢前，兒子還一邊翻看國小畢業紀念冊，鍾姓少年最近表現都很正常。
------------
文章原文：
12歲鍾姓少年疑因擔心上國中後課業壓力變大、不想上學，加上家庭經濟不好，曾被同學取笑，昨日開學日一大早，和小他1歲的妹妹到土城租屋處的12樓頂，隨即當著妹妹的面縱身跳下，送醫急救40分鐘仍不治，鐘父抱著兒子遺體痛哭失聲，悲戚場面令人動容。昨日上午7時許，即將進入國中校園的鍾姓少年，在起床後先向父親表示「有件事要做」，隨後以「幫我搬東西」為由，央請妹妹和他一起到頂樓。兄妹倆走上頂樓，鍾姓少年隨即爬上女兒牆，妹妹見狀大吃一驚趕緊問他：「你想幹嘛？」少年卻回答：「我想自殺」。當妹面輕生 父抱屍痛哭鍾妹見狀相當驚恐，趕忙問哥哥：「你要做什麼？是不是不想去上學？」「你不要這樣，不要跳下去！」少年回答：「妳說的是原因之一」，隨即就趁著大樓主委和妹妹要前去拉他時一躍而下，「碰！」一聲重重墜落在馬路邊機車停車格裡。鍾童雖立即被消防局送往亞東醫院急救，但他在到院前即無生命跡象，群醫急救40分鐘，仍無法挽回這條年輕生命。對於兒子選擇以輕生來拒絕開學，鍾父既不解又難過，他說，家裡經濟不好，但前晚他還帶著兒女一起到住家附近火鍋店吃飯，由於難得在外用餐，兄妹兩人都很開心，兒子回家後還主動幫忙掃地、拖地，完全看不出任何異狀。開心聚餐 事前無異狀鍾父表示，兒子雖曾在學校被同學取笑是低收入戶，他安慰兒子好好念書，以後不要像爸爸一樣，並沒有刻意要求課業表現，此後兒子都沒有出異狀。外傳鍾父一早與兒子爭執進而引發

## Practice

In [117]:
article1 = '''兆豐金涉洗錢案，董座徐光曦請辭，兆豐金控今天召開臨時董事決定由總經理吳漢卿暫代，但行政院長林全中午與民進黨立委便當會時向立委證實，已指派前第一金控董事長張兆順出任兆豐金控、兆豐銀行董事長。
林全今中午與立委午餐會，民進黨幹事長吳秉叡轉述說，林全告知立委，他已指派張兆順擔任兆豐金董事，今下午或明天會公佈。吳說，張有相關經驗又是會計師，對帳務查核各方面能力 ，相信希望能好好整頓兆豐金控整個狀況。
林全也告知立委，未來對兆豐金改革，包括公股代表、董監事 都會有一個特別跟過去不一樣的大幅度改變。（陳郁仁／台北報導）'''

article2 = '''Note 7驚傳電池爆炸，延後出貨恐阻礙行動部門下半年純益強勁復甦，衝擊三星電子股價今盤中一度重挫3.5%至2周低點156.3萬韓元，終場仍有2.2%跌幅，報158.4萬韓元。
三星電子最新智慧手機GALAXY Note 7上市以來買氣熱絡，預料將是蘋果新iPhone的勁敵。不過，就在新iPhone亮相前夕，GALAXY Note 7在全球傳出多起爆炸意外，變成名副其實的當紅「炸」子機，三星已疑似為此暫停GALAXY Note 7出貨，這對蘋果而言無疑是好消息。
GALAXY Note 7自8月24日以來已在全球發生5起充電時爆炸的意外。1名網友上周在南韓知名手機論壇ppomppu上傳多張GALAXY Note 7爆炸後的照片，表示手機是在用三星提供的充電器充電時爆炸。由於GALAXY Note 7在南韓售價為98.89萬韓元，南韓媒體戲稱GALAXY Note 7是「100萬韓元的炸彈」。
在GALAXY Note 7傳出多起爆炸意外後，三星電子昨向媒體表示，已暫停GALAXY Note 7出貨，原因是這支手機正在接受額外的品質檢驗測試。路透報導，三星未提到哪些國家的GALAXY Note 7出貨暫停，也未透露測試內容。《華爾街日報》則報導，三星已暫停南韓的GALAXY Note 7出貨，但拒絕透露其他國家的出貨是否也受影響。
另外，韓聯社昨在三星宣布GALAXY Note 7暫停出貨前報導，三星電子本周稍早已停止將GALAXY Note 7出貨給南韓電信商。（林文彬、于倩若／綜合外電報導）'''

article3 = '''【廣編特輯】
奶瓶是寶貝們時時接觸的好夥伴，所以挑選上需要十分注意。媽咪們就算再用心，奶瓶使用上仍有許多常見困擾：擔心奶瓶不好清潔、怕奶瓶遇酸鹼會釋放有毒物質、無法耐高溫、寶貝們不好拿或是無法吸吮得輕鬆舒適…。這些煩惱並不是難以避免，台灣培寶為了寶貝們的安全，悉心研發出一支讓媽媽們不再有選擇障礙的奶瓶！
使用全亞洲第一家耗資四億，引進最先進專業設備，生產最高規格
市面上奶瓶材質眾多讓消費者眼花撩亂，但你知道嗎？專家學者們一致認為，使用玻璃奶瓶其實是最安心且最安全的選擇。一支寶寶愛用媽咪放心的奶瓶，除了材質外，還是有其他要注意的小細節。因此台灣培寶使用投資四億引進最先進的專業設備，生產出最高規格的「培寶α33玻璃奶瓶」。身為全亞洲第一家用心研發的廠商，奶瓶的品質也通過了美國FDA、歐盟EN-14350等多項國際標準認證。
優秀奶瓶要具備的細節，你都注意到了嗎？
好的奶瓶不外乎在意寶寶使用的舒適度和安全性，一般產品卻很難兼顧。「培寶α33玻璃奶瓶」的橢圓瓶身具雙凹槽，讓寶寶手指好握不易滾動。超柔軟的奶嘴處也細心模擬母親乳房形狀，讓寶寶吸吮時舌頭自然蠕動，如同親餵般輕鬆舒適。媽咪使用方面，奶瓶的硼矽玻璃材質，耐高溫600°C外也耐溫差，可承受瞬間溫度變化150℃，遇酸鹼物質不會釋放有毒物。整支奶瓶光滑易清洗，較傳統鈉鈣玻璃及試管玻璃奶瓶更耐撞擊。'''
summary = [article1,article2,article3]

content=[]
for i in summary:
    content.append(' '.join(jieba.cut(i)))
    
# print content

## Summary 文章摘要

In [123]:
import re
setence1 = re.split("，|。|",article1)
setence2 = re.split("，|。|",article2)
# print setence1
# for i in setence2:
#     print i

all_map = []
avg_score = 0


# a=0
# for i in top_features:
#     print i,weight[0][indices[a]]
#     a=a+1

for sen in setence1:
    a=0
    score = 0
    for i in top_features:
        i = i.encode('utf-8')
        a=a+1       
        if i in sen:
            score += weight[0][indices[a]]
#     print score       
    avg_score += score
    data = {
        'sentence' : sen,
        'score' : score,
    }

    all_map.append(data)
sentenceSize = len(setence2)
avg_score = avg_score / sentenceSize
for i in all_map:        
    score = float(i['score'])
    if score > avg_score:
        print i['score'],i['sentence']

1.33725426998 兆豐金控今天召開臨時董事決定由總經理吳漢卿暫代
1.17009748623 但行政院長林全中午與民進黨立委便當會時向立委證實
1.2536758781 已指派前第一金控董事長張兆順出任兆豐金控、兆豐銀行董事長
0.835783918736 
林全今中午與立委午餐會
0.752205526863 林全告知立委
0.835783918736 他已指派張兆順擔任兆豐金董事
0.91936231061 相信希望能好好整頓兆豐金控整個狀況
0.752205526863 
林全也告知立委
0.835783918736 包括公股代表、董監事 都會有一個特別跟過去不一樣的大幅度改變
